# Data Analysis:
### This notebook aims to make an initial description of the data available for training in TrainingData_BU&Public_CWS.csv

This notebook:
- Replaces NaN values in 'verifier_user_id' field with -1
- Deletes the last row of the database since it doesn't contain any values. 
- Imports all the values in the database as the correct type (either string, float or int). 



TODO:
- export a cleaned version of the dataframe. 
- Investigate distribution of data. 
- Find out whether the clips are from different sections of a longer recordings.
- Find out whether there are duplicates within the database. This is very important for making sure that the validation set and the withheld test set do not contain examples which were also in the training set. This is a form of data leakage which would make the model seem to be performing better during evaluation than it would in reality. 


### Notes
There is a field 'ARU Task Status' with value of 'bad weather' and 'malfunction'. Option to filter out bad weather days. 






In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image

In [2]:
data_path = Path('../../data/raw/')

 # Load the database
 Reorder columns so that the clip_id is at the start. This way when the database is filtered, exported and re-loaded later we'll still have access to the original indices. 


In [3]:
meta = pd.read_csv(data_path/'TrainingData_BU&Public_CWS.csv')

meta['clip_id'] = meta.index

cols = list(meta.columns)
meta = meta[cols[-1:] + cols[:-1]]


/var/folders/hy/36n7jr6x3h74ddx507dc12_00000gn/T/ipykernel_16180/3376304062.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(data_path/'TrainingData_BU&Public_CWS.csv')


In [4]:
def display_all(df):
    with pd.option_context('display.max_columns', 70):
        with pd.option_context("display.max_rows", 1000):
            display(df)


# FIx NAs
1. Find out how many values in each field are blank

In [5]:
display_all(meta.tail())

,clip_id,organization,project,project_id,location,location_id,recording_date_time,recording_id,task_method,task_id,aru_task_status,species_code,species_common_name,species_scientific_name,species_class,detection_time,task_duration,tag_duration,min_tag_freq,max_tag_freq,tag_id,individual_order,vocalization,abundance,tag_rating,tag_is_verified,clip_channel_used,observer,observer_user_id,verifier_user_id,left_full_freq_tag_rms_peak_dbfs,left_full_freq_tag_rms_trough_dbfs,left_full_freq_tag_pk_count,left_full_freq_tag_dc_offset,left_full_freq_tag_min_level,left_full_freq_tag_max_level,left_full_freq_tag_peak_level_dbfs,left_freq_filter_tag_rms_peak_dbfs,left_freq_filter_tag_rms_trough_dbfs,left_freq_filter_tag_pk_count,left_freq_filter_tag_dc_offset,left_freq_filter_tag_min_level,left_freq_filter_tag_max_level,left_freq_filter_tag_peak_level_dbfs,right_full_freq_tag_rms_peak_dbfs,right_full_freq_tag_rms_trough_dbfs,right_full_freq_tag_pk_count,right_full_freq_tag_dc_offset,right_full_freq_tag_min_level,right_full_freq_tag_max_level,right_full_freq_tag_peak_level_dbfs,right_freq_filter_tag_rms_peak_dbfs,right_freq_filter_tag_rms_trough_dbfs,right_freq_filter_tag_pk_count,right_freq_filter_tag_dc_offset,right_freq_filter_tag_min_level,right_freq_filter_tag_max_level,right_freq_filter_tag_peak_level_dbfs,tagged_in_wildtrax,url,tag_spectrogram_url,clip_url,sensorId,tasks,status
1152835,1152835,BU,Zeller Woodpeckers- Time of Day,1241,Y-58-421-C,43958.0,2018-06-07 03:44:00,480766.0,None,442524.0,Transcribed,UNWO,Unidentified Woodpecker,NaN,Aves,42.90,60.0,0.93,0.15kHz,1.11kHz,2801910.0,1.0,Non-vocal,1,NaN,f,1.0,Austin Zeller,841.0,NaN,-42.91,-49.45,2.0,-0.000029,-0.021179,0.028168,-31.00,-48.50,-60.37,3.0,0.000010,-0.014435,0.010468,-36.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,3583,Active
1152836,1152836,BU,Zeller Woodpeckers- Time of Day,1241,Y-7-253-SW,303.0,2018-06-17 05:05:00,480207.0,None,441965.0,Transcribed,PIWO,Pileated Woodpecker,DRYOCOPUS PILEATUS,Aves,43.68,60.0,3.28,0.74kHz,2.85kHz,2796592.0,1.0,Call,1,NaN,f,1.0,Austin Zeller,841.0,NaN,-27.54,-35.22,3.0,0.033290,-0.008209,0.059052,-24.58,-59.69,-73.58,3.0,-0.000001,-0.017334,0.004333,-35.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,3583,Active
1152837,1152837,BU,Zeller Woodpeckers- Time of Day,1241,Y-7-253-SW,303.0,2018-06-17 06:05:00,480596.0,None,442354.0,Transcribed,PIWO,Pileated Woodpecker,DRYOCOPUS PILEATUS,Aves,16.30,60.0,2.34,0.18kHz,1.33kHz,2798296.0,1.0,Non-vocal,1,NaN,f,1.0,Austin Zeller,841.0,NaN,-24.51,-34.52,2.0,0.033245,-0.031403,0.101440,-19.88,-36.64,-46.33,5.0,-0.008149,-0.026703,0.008820,-31.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,3583,Active
1152838,1152838,BU,Zeller Woodpeckers- Time of Day,1241,Y-7-253-SW,303.0,2018-06-17 06:05:00,480596.0,None,442354.0,Transcribed,PIWO,Pileated Woodpecker,DRYOCOPUS PILEATUS,Aves,39.75,60.0,2.53,0.23kHz,1.33kHz,2798297.0,1.0,Song,1,NaN,f,1.0,Austin Zeller,841.0,NaN,-14.87,-35.70,2.0,0.033391,-0.174133,0.301270,-10.42,-32.44,-52.10,5.0,-0.004382,-0.042969,0.024841,-27.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,https://portal.wildtrax.ca/home/aru-tasks/reco...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,3583,Active
1152839,1152839,NaN,Community tagging: EMCLA Pilot Program 2012,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

The last entry in this .csv file contains NaNs for most of the fields. Delete this. 

In [6]:
meta.drop(meta.tail(1).index, inplace=True)

In [7]:
features = meta.columns
idx = 28
print(features[idx])
display_all(meta[features[idx]].value_counts(dropna=False))

observer_user_id


 18.0      147312
 15.0       70582
 41.0       68339
 22.0       60454
 1372.0     48288
 11.0       46565
 37.0       45935
 23.0       39339
 1349.0     31370
 1348.0     30879
 532.0      29471
 533.0      27144
 531.0      26341
 1343.0     24887
 527.0      24083
 26.0       22478
 20.0       20330
 55.0       20326
 1952.0     20090
 25.0       18431
 42.0       18019
 39.0       17679
 1057.0     16614
 1353.0     14988
 660.0      14182
 1357.0     13992
 17.0       11854
 1356.0     11448
 1208.0     10975
 31.0       10824
 1346.0     10083
 1881.0      9827
 67.0        9627
 2546.0      9503
 48.0        9287
 5.0         8476
 27.0        7712
 2047.0      7661
 1955.0      7555
 1128.0      6936
 1224.0      6527
 70.0        6011
 64.0        4886
 69.0        4815
 525.0       4571
 19.0        4515
 534.0       4441
 1269.0      4391
 1388.0      4284
 1234.0      3999
 1841.0      3826
 1275.0      3537
 1242.0      3262
 1257.0      3066
 71.0        3062
 1247.0   

Some of the ID columns contain a high number of 'NaN's. We can replace these with the value -1 

In [8]:
meta.dtypes

clip_id                 int64
organization           object
project                object
project_id              int64
location               object
                        ...  
tag_spectrogram_url    object
clip_url               object
sensorId               object
tasks                   int64
status                 object
Length: 65, dtype: object

Some of the data types have multiple types within one field. This is causing duplicate values when viewing data (e.g. the abundance field contains numeric values for 1 and a string values for 1). Different data types are are counted separately when using pandas.describe(). To fix this issue, specify the data type by passing in a type dictionary when reading the csv file. 

### Choose types for the different fields
If mixed types are required, change the value back to 'object'

In [9]:
type_dict = {
    "organization": str,
    "project": object,
    "project_id": int,
    "location": str,
    "location_id": int,
    "recording_date_time": object,
    "recording_id": int,
    "task_method": str,
    "task_id": int,
    "aru_task_status": str,
    "species_code": str,
    "species_common_name": str,
    "species_scientific_name": str,
    "species_class": str,
    "detection_time": float,
    "task_duration": float,
    "tag_duration": float,
    "min_tag_freq": object,
    "max_tag_freq": object,
    "tag_id": int,
    "individual_order": str,
    "vocalization": str,
    "abundance": str,
    "tag_rating": float,
    "tag_is_verified": object,
    "clip_channel_used": float,
    "observer": object,
    "observer_user_id": int,
    "verifier_user_id": int,
    "left_full_freq_tag_rms_peak_dbfs": float,
    "left_full_freq_tag_rms_trough_dbfs": float,
    "left_full_freq_tag_pk_count": float,
    "left_full_freq_tag_dc_offset": float,
    "left_full_freq_tag_min_level": float,
    "left_full_freq_tag_max_level": float,
    "left_full_freq_tag_peak_level_dbfs": float,
    "left_freq_filter_tag_rms_peak_dbfs": float,
    "left_freq_filter_tag_rms_trough_dbfs": float,
    "left_freq_filter_tag_pk_count": float,
    "left_freq_filter_tag_dc_offset": float,
    "left_freq_filter_tag_min_level": float,
    "left_freq_filter_tag_max_level": float,
    "left_freq_filter_tag_peak_level_dbfs": float,
    "right_full_freq_tag_rms_peak_dbfs": float,
    "right_full_freq_tag_rms_trough_dbfs": float,
    "right_full_freq_tag_pk_count": float,
    "right_full_freq_tag_dc_offset": float,
    "right_full_freq_tag_min_level": float,
    "right_full_freq_tag_max_level": float,
    "right_full_freq_tag_peak_level_dbfs": float,
    "right_freq_filter_tag_rms_peak_dbfs": float,
    "right_freq_filter_tag_rms_trough_dbfs": float,
    "right_freq_filter_tag_pk_count": float,
    "right_freq_filter_tag_dc_offset": float,
    "right_freq_filter_tag_min_level": float,
    "right_freq_filter_tag_max_level": float,
    "right_freq_filter_tag_peak_level_dbfs": float,
    "tagged_in_wildtrax": str,
    "url": str,
    "tag_spectrogram_url": str,
    "clip_url": str,
    "sensorId": str,
    "tasks": int,
    "status": str,
}

The column at index 28 is verifier user id. It contains a lot of NaN values. These are replaced with -1.

In [10]:
column_with_nas = meta.columns[2]
column_with_nas

'observer_user_id'

In [11]:
meta.verifier_user_id[0], sum(meta.verifier_user_id.isna()

SyntaxError: incomplete input (626098822.py, line 1)

In [ ]:
meta.loc[meta['verifier_user_id'].isna(), 'verifier_user_id'] = -1

In [ ]:
meta.verifier_user_id[0], sum(meta.verifier_user_id.isna())

## Change all the data types in the dataframe to the chosen types contained in the type dict.

In [ ]:
meta = meta.astype(type_dict)

# Look at the reording ids

In [ ]:
recording_ids = meta.recording_id.value_counts()
meta.recording_id.value_counts().head()

In [ ]:
top_recordings = meta.recording_id.value_counts()

## List of the recording IDs which generate the most clips. 
The recording ids aren't unique, indicating that there are mulitple references to the same recording within the database.


In [ ]:
top_recordings

In [ ]:
ax = sns.scatterplot(x=top_recordings.index, y=top_recordings.values)
plt.setp(ax.collections, alpha=.5, linewidth=.1)
plt.xlabel('recording_id')
plt.ylabel('number_of_clips')
plt.title('Number of clips per recording in the database')

List of the different projects which contribute to the recordings in the database. 

In [ ]:
projects = meta.project.unique().tolist()
projects

# Filter out old recordings. 
There are 114 projects in the database. Some of the older (pre-2016 - 2017) recording units had a high signal to noise ratio due to the recording devices used. Filtering out projects older than this could improve average quality of recordings. 
<br><br>
Also more recent data might better represent current call patterns in the field. 


In [ ]:
years = range(2017, 2024)

In [ ]:
recent_projects = []
for year in years:
    year_projects = [project for project in projects if f'{year}' in project]
    recent_projects.extend(year_projects)

## Projects filtered by most recent years

In [ ]:
recent_projects

In [ ]:
len(meta.recording_id.unique())

There are 97439 unique recordings in the database.


### Remove the clips which don't contain a link to a recording. 
These won't be useful since we can't download the audio for these clips. 

In [ ]:
meta.drop(meta.loc[meta.clip_url=='nan'].index, inplace=True)
meta.clip_url.value_counts()

In [ ]:
meta.loc[meta.species_code=='OSFL'].url.iloc[1]

# Look at the distribution of clip lengths in  TrainingData_BU&Public_CWS.csv

In [ ]:
df = meta.loc[meta.tag_duration<30]
fig, ax = plt.subplots()
sns.histplot(data=df, x='tag_duration', ax=ax, bins=50)
ax.set_xlabel('Duration (s)')
ax.set_ylabel('Count')
ax.set_title('Sample lengths for samples < 30s')
plt.show()

In [ ]:
ax = sns.histplot(data=meta.loc[meta.tag_duration > 40], x='tag_duration', bins=20)
ax.set_xlabel('Duration (s)')
ax.set_ylabel('Count')
ax.set_title('Sample lengths for samples > 40s')
plt.show()

# Look at the data for OSFL recordings. 

In [ ]:
osfl_df = meta.loc[meta.species_code=='OSFL']
display_all(osfl_df.head())

## Here are the projects which generated olive sided flycatcher clips since 2017

In [ ]:
osfl_projects = osfl_df.project.unique()
len(osfl_projects)

In [ ]:
recent_osfl_projects = []
for year in years:
    year_projects = [project for project in osfl_projects if f'{year}' in project]
    recent_osfl_projects.extend(year_projects)

In [ ]:
recent_osfl_projects, len(recent_osfl_projects)

# Look at the reording ids of recordings which have the most OSFL tags. 

In [ ]:
osfl_recording_ids = osfl_df.recording_id.value_counts()
osfl_df.recording_id.value_counts().head()

In [ ]:
top_recordings = osfl_df.recording_id.value_counts()

## List IDs which generate the most clips. 
The recording ids aren't unique, indicating that there are mulitple references to the same recording within the database.

In [ ]:
top_recordings

In [ ]:
ax = sns.scatterplot(x=top_recordings.index, y=top_recordings.values)
plt.setp(ax.collections, alpha=.5, linewidth=.1)
plt.xlabel('recording_id')
plt.ylabel('number_of_clips')
plt.title('Number of clips per recording in the database')
plt.show()

One of the recordings contains 38 olive sided flycatcher calls. Let's verify this on wildtrax by taking a listen there. 

In [ ]:
osfl_df[osfl_df.recording_id==218582].url[896079]

The index 896079 is the index of one of the clips from recording 218582. Inputted here to reveal the full URL rather than truncated list. 

In [ ]:
images = Path('../../references/screenshots')
display(Image(images/'img1.png'))

In [ ]:
display(Image(images/'clip1.png'))

- This single 60 second recording does indeed contain 38 separate olive sided flycatcher calls. 
- There are a lot of other bird calls in the background. 
- Some of the flycatcher calls can be seen in the spectrogram on WildTrax. 
- Others are so densely populated with calls that it is hard to make out the shape.

# Building the datasets. 
### osfl_call
We need to build two datasets to train a CNN recognizer. The first needs to be made up of recordings containing an olive sided flycatcher call. These will be easy to obtain, since the tagging method used on WildTrax contains a tag for OSFL. We have access to human labelled data from a variety of projects, and URLs links directly to AWS hosted audio files of pre-cut clips. 

### no_osfl_call
The second datased needs to contain clips of the same duration, and from the same recordings as the first This second dataset will be harder to curate, since there is no specific tag for 'not an olive sided flycatcher' in the database. To curate this dataset will require knowledhe and exploration of the tagging methods used for the different wildtrax projects, and some creativity. 

Here are two ideas to build the datasets.
### Option 1 - From 60s no restrictions tag type:
- Obtain full length recordings (not just tags/clips) for recordings which have been tagged using the 60s no restrictions method. Then cut the gaps between OSFL tags and use these as no-call data. 

The disadvantage of using this technique would be that it can't be used for the projects which just tag the first vocalization of a species in a recording, or in a given time period. 

### Option 2 - From the portion of the recording before the first tagged call
- Obtain full length recordings for recordings which have been tagged using the '1SPT' or '1SPM' task method, and treat the audio before the first tag as audio which does not contain an olive sided flycatcher call. 

Advantages - we can build a dataset with call and no-call samples from the same audio recording no matter which task method was used.

Disadvantages - but for the no-call we can only choose from a potentially short portion of recording before the first vocalization. We can not assume that audio after the first olive sided flycatcher ID is absent of any calls of this species. 

### mis-labelled clips
There will inevitably be some missed or mis-labelled tags, and these will need to be identified during model training. Some amount of manual checks might be necessary. 

__Using the model's predictions to clean the dataset__:
Training an initial model early, then looking at where it confidently predicts the incorrect class can be a quick way of cleaning or re-labeling mis-labelled data from the dataset.



In [ ]:
# clip 896079 project ID
osfl_df[osfl_df.recording_id==218582].project[896079]

The screenshot above is taken from a project named _Lebeuf-Taylor - BRCR, RCKI, OSFL, REVI, TEWA, YRWA - BU 2021_ in our .csv file. The sampling method on wildtrax is stated as "__60s no restrictions__" which unfortunately hasn't transferred over to the .csv file, which just says "None" for this task method.

In [ ]:
# Available tagging methods within this .csv file
osfl_df.task_method.unique()

In [ ]:
# clip 896079 sample method
osfl_df[osfl_df.recording_id==218582].clip_url[896079]

# Export the cleaned version of the database

In [ ]:
processed_data_path = Path('../../data/processed/')

In [ ]:
meta.to_csv(processed_data_path/'processed_metadata.csv')

In [ ]:
meta.to_pickle(processed_data_path/'processed_metadata.pkl')

# Load data from other sources:
### Look at the BU everything forever csv

In [ ]:
!ls $data_path/SpeciesRawDownload

In [ ]:
everythingforever = pd.read_csv(data_path/'SpeciesRawDownload/recording_report.csv')

In [ ]:
everythingforever.task_url[0]